In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('C:/users/LL/Documents/GitHub/AMLF_projects/data.csv')

In [3]:
df = df.drop(columns = ['Unnamed: 0'])

In [4]:
df.head()

,Date,permno,return,mom1m,mom12m,chmom,indmom,mom36m,turn,mvel1,...,baspread,retvol,idiovol,beta,betasq,ep,sp,agr,nincr,return(t-1)
0,2001-01-31,13610,-0.202242,0.277978,-0.082232,0.518490,0.198455,-0.258322,0.820391,9.897840e+05,...,0.037278,0.032888,0.056769,0.398706,0.158967,0.019041,1.472909,0.325935,5.0,NaN
1,2001-01-31,13856,-0.112756,0.095372,0.300233,-0.147620,0.069669,0.000718,0.660636,7.152231e+07,...,0.033983,0.022389,0.042020,0.106574,0.011358,0.039970,0.397105,0.225463,1.0,NaN
2,2001-01-31,13901,0.010566,0.166088,0.759861,0.504130,0.400659,-0.440929,0.775189,9.783550e+07,...,0.032844,0.023475,0.050243,0.088382,0.007811,0.142695,1.148088,-0.024383,2.0,NaN
3,2001-01-31,13928,0.033114,0.006637,0.236486,0.039925,0.400659,0.025686,0.813903,1.451888e+07,...,0.035964,0.023917,0.037427,0.159983,0.025595,0.051091,1.138525,-0.069288,1.0,NaN
4,2001-01-31,13936,0.014335,0.009709,0.574141,0.224657,-0.075486,-0.397110,0.755288,3.550430e+05,...,0.031413,0.030343,0.068491,1.060490,1.124640,0.091598,6.902488,0.000838,0.0,NaN


In [5]:
print(len(df.columns))
print(df.columns)

23
Index(['Date', 'permno', 'return', 'mom1m', 'mom12m', 'chmom', 'indmom',
       'mom36m', 'turn', 'mvel1', 'dolvol', 'ill', 'zerotrade', 'baspread',
       'retvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr',
       'return(t-1)'],
      dtype='object')


In [6]:
# df.dtypes

According to note 30: "Therefore, to predict returns at month t+1, we use most recent monthly characteristics at the end of month t." <br>
Hence, **shift return t+1 to serve as response: r(t+1)**.

In [7]:
df['r(t+1)'] = df.groupby('permno')['return'].shift(-1)

In [8]:
len(df)

114000

In [9]:
# df.isna().sum()

### handle missing data

According to note 30 (bottom of p 2248): "Another issue is missing characteristics, which we replace with the cross-sectional median at each month for each stock, respectively." <br>
Hence, calculate monthly cross-sectional median for features: **'mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr'**.

In [10]:
df_filled = df.copy()
for feature in ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']:
    df_filled[feature] = df_filled.groupby('Date')[feature].transform(lambda x: x.fillna(x.median()))

In [11]:
# df_filled.isna().sum()

In [12]:
df.loc[:, ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']] = df_filled.loc[:,['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']]

In [13]:
df.isna().sum()

Date             0
permno           0
return           0
mom1m            0
mom12m           0
chmom            0
indmom           0
mom36m           0
turn             0
mvel1            0
dolvol           0
ill              0
zerotrade        0
baspread         0
retvol           0
idiovol          0
beta             0
betasq           0
ep               0
sp               0
agr              0
nincr            0
return(t-1)    500
r(t+1)         500
dtype: int64

In [14]:
df['Date'] = pd.to_datetime(df['Date'])

# Set the datetime column as index
df.set_index('Date', inplace=True, drop = True)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scaled = scaler.fit_transform(df)

df_scaled = pd.DataFrame(df_scaled, columns=df.columns)


In [16]:
permno = df['permno'].reset_index(drop = True)

In [17]:
df_scaled['permno'] = permno

In [18]:
df_scaled.index = df.index

In [19]:
df_scaled_2 = df_scaled.drop(columns = [ 'permno', 'return'])

### importing custom functions

In [37]:
%run -i "C:/users/LL/Documents/GitHub/AMLF_projects/pj1_functions.ipynb"

### spliting data

**generate rolling windows**

In [24]:
# generate rolling samples

windows = generate_rolling_windows(df_scaled_2, 19, 6, 4)

In [25]:
len(windows)

8

In [ ]:
for window in windows:
    mean = window[0]['r(t+1)'].mean()
    print(mean)

### subsampling data
top and bottom 100 stocks on market value (mvel1)**

**get top 100 and bot 100 of all years**

In [ ]:
subset = df_scaled.copy()

In [ ]:
mvel_sorted_top = subset['2019-12-01':].sort_values('mvel1',ascending=False).head(100).reset_index(drop=True)
filter_values_top = mvel_sorted_top['permno']
top_100 = subset[subset['permno'].isin(filter_values_top)]


mvel_sorted_bot = subset['2019-12-01':].sort_values('mvel1',ascending=False).tail(100).reset_index(drop=True)
filter_values_bot = mvel_sorted_bot['permno']
bot_100 = subset[subset['permno'].isin(filter_values_bot)]


In [ ]:
top_100.shape
# 19*12*10 = 2280

**splitting data & separating X and y** <br>
splitting data 10:9 years - to make the training set the same size as the training and validation set combined, leave testing set out of sample

In [ ]:
# top_windows = generate_rolling_windows(top_100.dropna(), 19, 6, 4)
# bot_windows = generate_rolling_windows(bot_100.dropna(), 19, 6, 4)

In [ ]:
# len(top_windows)

In [ ]:
# len(bot_windows)

In [ ]:
training_top = top_100[:'2011-01-01'].dropna()
testing_top = top_100['2011-01-02':].dropna()

In [ ]:
training_bot = bot_100[:'2011-01-01'].dropna()
testing_bot = bot_100['2011-01-02':].dropna()

separating X and y

In [ ]:
X_test_top = testing_top.drop(columns = ['permno', 'return', 'r(t+1)'])
y_test_top = testing_top['r(t+1)']

X_test_bot = testing_bot.drop(columns = ['permno', 'return', 'r(t+1)'])
y_test_bot = testing_bot['r(t+1)']

# Enet


prediction: g*(z i,t) - depends on neither i nor t, is dependend on z only through z i,t <br>
responses: r i, t+1


In [26]:
from sklearn.linear_model import ElasticNet

**tuning**

1. set-up the search grid
2. tuning: recursive rolling validation set w/ oos R^2 metric -- <br>
    roll one unit froward each time and take the average as the final score <br>
3. pick the best hyperparameters, retrain the model

In [27]:
# set up searching grid

alpha_values = [0.4, 0.5, 0.6]
l1_ratio_values = np.logspace(-4, -1, 10)

param_enet = [{'alpha': alpha, 'l1_ratio': l1_ratio} for alpha in alpha_values for l1_ratio in l1_ratio_values]

# for param_combination in param_combinations:
#     print(param_combination)

In [28]:
# the model

Enet = ElasticNet()

In [38]:
results = []
for window in windows:
    
    #apply tuning function
    result = tuning_1(Enet, param_enet, window[0].drop(columns = ['r(t+1)']), window[0]['r(t+1)'], window[1].drop(columns = ['r(t+1)']), window[1]['r(t+1)'])
    
    # print out tuning parameters of eah]ch split
    print(f"the optimal parameters are {result[0]} and the best score is {result[1]}.")
    
    results.append(result)

the optimal parameters are {'alpha': 0.6, 'l1_ratio': 0.1} and the best score is 0.0007755796797865866.
the optimal parameters are {'alpha': 0.6, 'l1_ratio': 0.1} and the best score is 0.002077004617268674.
the optimal parameters are {'alpha': 0.6, 'l1_ratio': 0.1} and the best score is -0.0014524607777255394.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.0063834100482236256.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.004957498963627627.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.0035499187056395876.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.0012202180037492738.
the optimal parameters are {'alpha': 0.4, 'l1_ratio': 0.0001} and the best score is -0.0022601722569968175.


**best model** <br>
<br>
Fit on all data in training and validation sets.

In [40]:
Enet_opt = ElasticNet()
r2_Enet = []
# r2_Enet_top = []
# r2_Enet_bot = []
for i in range(8):
    
    for param_name, param_value in results[i][0].items():
     setattr(Enet_opt, param_name, param_value)
    
    Enet_opt.fit(windows[i][3].drop(columns = ['r(t+1)']), windows[i][3]['r(t+1)'])
    y_pred_enet = Enet_opt.predict(windows[i][2].drop(columns = ['r(t+1)']))
    
    r2_enet = r2_score_wo_demeaning(windows[i][2]['r(t+1)'], y_pred_enet)
    r2_Enet.append(r2_enet)       # fit best model for each split
    
    # used model to generate prediction for the top 100
#     y_pred_enet_top = Enet_opt.predict(X_test_top)
#     r2_enet_top = r2_score_wo_demeaning(y_test_top, y_pred_enet_top)
#     r2_Enet_top.append(r2_enet_top)
    
#     y_pred_enet_bot = Enet_opt.predict(X_test_bot)
#     r2_enet_bot = r2_score_wo_demeaning(y_test_bot, y_pred_enet_bot)
#     r2_Enet_bot.append(r2_enet_bot)

In [41]:
# all the best r^2-oos for 8 splits stored in one list:

r2_Enet

[0.0005663579684225262,
 0.0007243052297416508,
 0.00015885305985385845,
 -0.001960710816258926,
 -0.0008903639645345685,
 2.6038543099682343e-05,
 -0.002316165787667135,
 -0.00016025654555895663]

In [42]:
#take average

print("R^2-oos:", sum(r2_Enet) / len(r2_Enet))

R^2-oos: -0.0004814927891127335


**top and bottom 100 prediction**

In [ ]:
# r2_Enet_top

In [ ]:
# take average

# print("Top 100 stocks R^2-oos:", sum(r2_Enet_top) / len(r2_Enet_top))

In [ ]:
# r2_Enet_bot

In [ ]:
# take average

# print("Bottom 100 stocks R^2-oos:", sum(r2_Enet_bot) / len(r2_Enet_bot))

**feature importance**

In [ ]:
# import matplotlib.pyplot as plt

# coefficients = Enet_opt.coef_

# # Get feature names
# # Assuming feature_names is a list of your feature names
# feature_names = X_train_combined.columns  # Insert your feature names here

# # Create a dictionary to store feature importance scores with feature names
# feature_importance_dict = dict(zip(feature_names, np.abs(coefficients)))

# # Sort feature importance dictionary by importance score
# sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# # Extract feature names and importance scores after sorting
# sorted_feature_names = [x[0] for x in sorted_feature_importance]
# sorted_feature_importance_scores = [x[1] for x in sorted_feature_importance]

# # Plotting
# plt.figure(figsize=(8, 6))
# plt.barh(sorted_feature_names, sorted_feature_importance_scores)
# plt.xlabel('Feature Importance')
# plt.ylabel('Features')
# plt.title('Feature Importance of Elastic Net Model')
# plt.gca().invert_yaxis()  # Invert y-axis to have the most important features at the top
# plt.show()

# NN

In [ ]:
import keras
from keras import layers
from keras import Sequential

## model

All activation functions are ReLU function <br>
optimizer: SGD w/ learning rate shrinkage: adam <br>

set-up grid

In [47]:
nepoch_val = [25, 50, 75, 100]
lr_val = [0.05, 0.01, 0.001]
nbatch_val = [1500, 2500, 3500]

param_nn = [{'epoch': epoch, 'learning_rate': learning_rate, 'batch_size': batch_size} for epoch in nepoch_val for learning_rate in lr_val for batch_size in nbatch_val]

### 1-layer

In [48]:
input_dim = 20
layer1_n = 32

model_1 = Sequential([
            layers.Dense(layer1_n, activation='relu', input_dim=input_dim),
            layers.Dense(1, activation='linear')
        ])

**tuning**

In [ ]:
results_nn1 = []
opt_scores_nn1 = []
opt_paras_nn1 = []

for window in windows:
    print("start running")
    # to arry
    X_train_nn = np.asarray(window[0].drop(columns = ['r(t+1)']).values)
    y_train_nn = np.asarray(window[0]['r(t+1)'].values)

    X_val_nn = np.asarray(window[1].drop(columns = ['r(t+1)']).values)
    y_val_nn = np.asarray(window[1]['r(t+1)'].values)
    
    # apply function
    result_nn1 = compile_and_tune_model(model_1, param_nn, X_train_nn, y_train_nn, X_val_nn, y_val_nn)
    
    # get score
    result_nn1 = pd.DataFrame(result_nn1).sort_values('val_r2_score_wo_demeaning_nn',ascending=False)
    print('finished')
    opt_score = result_nn1.iloc[0,1]
    print("opt_score: ", opt_score)
    opt_para = result_nn1.iloc[0,0]
    print("opt_result: ", opt_para)
    # results_nn1.append(result_nn1.iloc[0,])
    opt_scores_nn1.append(opt_score)
    opt_paras_nn1.append(opt_para)

In [ ]:
result_nn1

**best model**

In [ ]:
model_nn1 = Sequential([
            layers.Dense(layer1_n, activation='relu', input_dim=input_dim),
            layers.Dense(1, activation='linear')
        ])

last_epoch_metrics = []
r2_nn1_tops = []
r2_nn1_bots = []

for i in range(8):
    
    model_nn1.compile(optimizer=keras.optimizers.Adam(learning_rate=opt_paras_nn1[i]['learning_rate']),
                          loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)

    X_train_combined_nn = np.asarray(windows[i][3].drop(columns = ['r(t+1)']).values)
    y_train_combined_nn = np.asarray(windows[i][3]['r(t+1)'].values)
    
    X_test_nn = np.asarray(windows[i][2].drop(columns = ['r(t+1)']).values)
    y_test_nn = np.asarray(windows[i][2]['r(t+1)'].values)
    
    # fit the model
    print('start fitting')
    history = model_nn1.fit(X_train_combined_nn, y_train_combined_nn, epochs=opt_paras_nn1[i]['epoch'], batch_size=opt_paras_nn1[i]['batch_size'],
                                validation_data=(X_test_nn, y_test_nn), verbose=0)
    print('finished fitting')
    
    # get scores
    last_epoch_metric = history.history['val_r2_score_wo_demeaning_nn'][-1]
    last_epoch_metrics.append(last_epoch_metric)
    print('best score: ', last_epoch_metric)
    
    # top 100
    y_pred_nn1_top = model_nn1.predict(X_test_top)
    r2_nn1_top = r2_score_wo_demeaning(y_test_top, y_pred_nn1_top)
    r2_nn1_tops.append(r2_nn1_top)
    print('top best score: ', r2_nn1_top)
    
    # bot 100
    y_pred_nn1_bot = model_nn1.predict(X_test_bot)
    r2_nn1_bot = r2_score_wo_demeaning(y_test_bot, y_pred_nn1_bot)
    r2_nn1_bots.append(r2_nn1_bot)
    print('bot best score: ', r2_nn1_bot)

In [ ]:
last_epoch_metrics

In [ ]:
print("NN1 average R^2-oos:", sum(last_epoch_metrics) / len(last_epoch_metrics))

**top and bottom 100 prediction**

In [ ]:
r2_nn1_tops

In [ ]:
print("Top 100 stocks R^2-oos:", sum(r2_nn1_tops) / len(r2_nn1_tops))

In [ ]:
r2_nn1_bots

In [ ]:
print("Bottom 100 stocks R^2-oos:", sum(r2_nn1_bots) / len(r2_nn1_bots))

### 3-layer

In [ ]:
input_dim = 20
layer1_n = 32
layer2_n = 16
layer3_n = 8


model_3 = Sequential([
            layers.Dense(layer1_n, input_dim = input_dim, activation='relu'),
            layers.Dense(layer2_n, activation='relu'),
            layers.Dense(layer3_n, activation='relu'),
            layers.Dense(1, activation='linear')
        ])


**tuning**

In [ ]:
results_nn3 = []
opt_scores_nn3 = []
opt_paras_nn3 = []

for window in windows:
    print('start running')
    # to arry
    X_train_nn = np.asarray(window[0].drop(columns = ['r(t+1)']).values)
    y_train_nn = np.asarray(window[0]['r(t+1)'].values)

    X_val_nn = np.asarray(window[1].drop(columns = ['r(t+1)']).values)
    y_val_nn = np.asarray(window[1]['r(t+1)'].values)

    result_nn3 = compile_and_tune_model(model_3, param_nn, X_train_nn, y_train_nn, X_val_nn, y_val_nn)
    print('finished running')
    
    # get score
    result_nn3 = pd.DataFrame(result_nn3).sort_values('val_r2_score_wo_demeaning_nn',ascending=False)
    opt_score = result_nn3.iloc[0,1]
    print('opt_score: ', opt_score)
    opt_para = result_nn3.iloc[0,0]
    print('opt_para: ', opt_para)
    # results_nn1.append(result_nn1.iloc[0,])
    opt_scores_nn3.append(opt_score)
    opt_paras_nn3.append(opt_para)

In [ ]:
result_nn3

check one by one

In [ ]:
# 4th split
# 2001-2013; 2014-2019
parameters = {'epoch': 25, 'learning_rate': 0.01, 'batch_size': 3000}

In [ ]:
X_tr4 = windows[3][3].drop(columns = ['r(t+1)'])
y_tr4 = windows[3][3]['r(t+1)']

X_ts4 = windows[3][2].drop(columns = ['r(t+1)'])
y_ts4 = windows[3][2]['r(t+1)']

In [ ]:
model_nn3_4 = Sequential([
            layers.Dense(32, activation='relu', input_dim=20),
            layers.Dense(16, activation='relu'),
            layers.Dense(8, activation='relu'),
            layers.Dense(1, activation='linear')
        ])

print('start fitting')

model_nn3_4.compile(optimizer=keras.optimizers.Adam(learning_rate=parameters['learning_rate']),
                          loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)

X_train_combined_nn3_4 = np.asarray(X_tr4.values)
y_train_combined_nn3_4 = np.asarray(y_tr4.values)

X_test_nn3_4 = np.asarray(X_ts4.values)
y_test_nn3_4 = np.asarray(y_ts4.values)
    
outcome = model_nn3_4.fit(X_train_combined_nn3_4, y_train_combined_nn3_4, epochs=parameters['epoch'], batch_size=parameters['batch_size'],verbose=0)
print('finished')
# get scores
# R2_nn3_4 = outcome.history['val_r2_score_wo_demeaning_nn'][-1]
# print('best score: ', R2_nn3_4)

pred_nn4 = model_nn3_4.predict(X_test_nn3_4)
score = r2_score_wo_demeaning(y_test_nn3_4, pred_nn4)
print(score)

**best model**

In [ ]:
model_nn3 = Sequential([
            layers.Dense(layer1_n, activation='relu', input_dim=input_dim),
            layers.Dense(layer2_n, activation='relu'),
            layers.Dense(layer3_n, activation='relu'),
            layers.Dense(1, activation='linear')
        ])

last_epoch_metrics_nn3 = []
r2_nn3_tops = []
r2_nn3_bots = []

for i in range(8):
    print('start running')
    model_nn3.compile(optimizer=keras.optimizers.Adam(learning_rate=opt_para_nn3[i]['learning_rate']),
                          loss='mean_squared_error', metrics=r2_score_wo_demeaning_nn)

    X_train_combined_nn = np.asarray(windows[i][3].drop(columns = ['r(t+1)']).values)
    y_train_combined_nn = np.asarray(windows[i][3]['r(t+1)'].values)
    
    X_test_nn = np.asarray(windows[i][2].drop(columns = ['r(t+1)']).values)
    y_test_nn = np.asarray(windows[i][2]['r(t+1)'].values)
    
    # fit the model
    history = model_nn3.fit(X_train_combined_nn, y_train_combined_nn, epochs=opt_paras_nn3[i]['epoch'], batch_size=opt_paras_nn3[i]['batch_size'],
                                validation_data=(X_test_nn, y_test_nn), verbose=0)
    print('finished')
    # get scores
    last_epoch_metric = history.history['val_r2_score_wo_demeaning_nn'][-1]
    last_epoch_metrics_nn3.append(last_epoch_metric)
    print('best score: ', last_epoch_metric)
    
    # top 100
    y_pred_nn3_top = model_nn3.predict(X_test_top)
    r2_nn3_top = r2_score_wo_demeaning(y_test_top, y_pred_nn3_top)
    r2_nn3_tops.append(r2_nn3_top)
    print('top best: ', r2_nn3_top)
    
    # bot 100
    y_pred_nn3_bot = model_nn3.predict(X_test_bot)
    r2_nn3_bot = r2_score_wo_demeaning(y_test_bot, y_pred_nn3_bot)
    r2_nn3_bots.append(r2_nn3_bot)
    print('bot best: ', r2_nn3_bot)

In [ ]:
last_epoch_metrics_nn3

In [ ]:
print("NN3 average R^2-oos:", sum(last_epoch_metrics_nn3) / len(last_epoch_metrics_nn3))

**top and bottom 100 prediction**

In [ ]:
r2_nn3_tops

In [ ]:
print("Top 100 stocks R^2-oos:", sum(r2_nn3_tops) / len(r2_nn3_tops))

In [ ]:
r2_nn3_bots

In [ ]:
print("Bottom 100 stocks R^2-oos:", sum(r2_nn3_bots) / len(r2_nn3_bots))